In [28]:
import numpy as np
import pandas as pd
import pyodbc

In [29]:
years = list(range(89, 99))+ [99] + list(range(1400, 1403))
#years = [89]
for year in years:
    # Path to your .accdb file
    file_path = fr'C:\Users\Mahsa\Desktop\geo\raw_data\{year}\Outfile-{year}.accdb'
    # Connection string (for 64-bit Access Database Engine)
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        f'DBQ={file_path};'
    )
    # Connect
    conn = pyodbc.connect(conn_str)
    # List all table names
    tables = [table.table_name for table in conn.cursor().tables(tableType='TABLE')]
    print(f"Year is :{str(year)}".center(50))
    #---------------------------------------------------------------------------------
    Rdata = pd.read_sql(f"SELECT * FROM R{year}P1", conn)
    Udata = pd.read_sql(f"select * from U{year}P1", conn)
    data = pd.concat([Rdata, Udata], axis=0)
    
    data.rename(
        columns = {
            "Address": "HHID",
            "DYCOL01": "IndiNum",
            "DYCOL03": "RelationHead",
            "DYCOL04": "Sex",
            "DYCOL05": "Age",
            "DYCOL06": "Literacy",
            "DYCOL07": "Current_student",
            "DYCOL08": "EducationLevel",
            "DYCOL09": "EmploymentStatus",
            "DYCOL10": "MaritalState"   
        },
        inplace= True
    )
    data["HHID"] = data["HHID"].astype("str")
    mask = data["HHID"].str.contains(r"\.")
    data.loc[mask, 'HHID'] = data.loc[mask, "HHID"].str.split(".").str[0]
    data = data.loc[data["HHID"].notna()]
    
    data["IndiNum"] = data["IndiNum"].astype("str")
    mask = data["IndiNum"].str.contains(r"\.")
    data.loc[mask, 'IndiNum'] = data.loc[mask, "IndiNum"].str.split(".").str[0]
    data["IndiNum"] = data["IndiNum"].str.pad(2,'left',"0")
    
    data["EducationLevel"] = pd.to_numeric(data["EducationLevel"], errors='coerce')
    data["RelationHead"] = pd.to_numeric(data["RelationHead"], errors='coerce')
    data["Sex"] = pd.to_numeric(data["Sex"], errors='coerce')
    data["Age"] = pd.to_numeric(data["Age"], errors='coerce')
    data["Literacy"] = pd.to_numeric(data["Literacy"], errors='coerce')
    data["Current_student"] = pd.to_numeric(data["Current_student"], errors='coerce')
    data["EmploymentStatus"] = pd.to_numeric(data["EmploymentStatus"], errors='coerce')
    data["MaritalState"] = pd.to_numeric(data["MaritalState"], errors='coerce')
    
    data["RelationHead"] = np.where(
        data["RelationHead"] == 1, "Head",np.where(
            data["RelationHead"] == 2, "Spouse", np.where(
                data["RelationHead"] == 3, "Child", np.where(
                    data["RelationHead"] == 4, "s/d in low", np.where(
                        data["RelationHead"] == 5, "Grandchild", np.where(
                            data["RelationHead"] == 6, "Mom/Dad", np.where(
                                data["RelationHead"] == 7, "Bro/Sis", np.where(
                                    data["RelationHead"] == 8, "OtherRelative", np.where(
                                        data["RelationHead"] == 9, "NoneRe", pd.NA
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
    data["Sex"] = np.where(
        data["Sex"]==1, "Male", np.where(
            data["Sex"] == 2, "Female", pd.NA
        )
    )
    data["Literacy"] = np.where(
        data["Literacy"] == 1, "Literate", np.where(
            data["Literacy"] == 2, "Illiterate", pd.NA
        )
    )
    data["Current_student"] = np.where(
        data["Current_student"]== 1, "Studying", np.where(
            data["Current_student"] == 2, "NotStudying", pd.NA
        )
    )
    data["EmploymentStatus"] = np.where(
        data["EmploymentStatus"] == 1, "Employed", np.where(
            data["EmploymentStatus"]== 2, "Unemployed", np.where(
                data["EmploymentStatus"]== 3, "IncomeWithoutJob", np.where(
                    data["EmploymentStatus"]==4, "Student", np.where(
                        data["EmploymentStatus"]== 5, "Housekeeper", np.where(
                            data["EmploymentStatus"]== 6, "Other", pd.NA
                        )
                    )
                )
            )
        )
    )
    data["MaritalState"] = np.where(
        data["MaritalState"]==1, "Married", np.where(
            data["MaritalState"]==2, "Widow", np.where(
                data["MaritalState"]==3, "Divorced", np.where(
                    data["MaritalState"]==4, "NotMarried", pd.NA
                )
            )
        )
    )
    
    print(
        data.apply(lambda col: col.unique()),
        f"The number of Rows is : {data.shape[0]}",       
    )
    print(data.loc[data["Literacy"].isna(),"Age"].max()<7)
    print(data.loc[(data["Current_student"].isna()) & (data["Literacy"].notna()),"Literacy"].nunique()==1)
    print(
    ((data["Literacy"].isna()) | (data["Literacy"] == "Illiterate")).sum() == data["Current_student"].isna().sum()
    )
    data.to_pickle(fr"C:\Users\Mahsa\Desktop\geo\base2\P1_{year}")
    

                   Year is :89                    


C:\Users\Mahsa\AppData\Local\Temp\ipykernel_14816\3135642001.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Rdata = pd.read_sql(f"SELECT * FROM R{year}P1", conn)
C:\Users\Mahsa\AppData\Local\Temp\ipykernel_14816\3135642001.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Udata = pd.read_sql(f"select * from U{year}P1", conn)


HHID                [2000121311, 2000121315, 2000001152, 200000114...
IndiNum             [02, 03, 04, 01, 05, 06, 07, 08, 09, 10, 11, 1...
RelationHead        [Spouse, Child, Head, Bro/Sis, Mom/Dad, s/d in...
Sex                                                    [Female, Male]
Age                 [35, 11, 0, 42, 36, 17, 12, 40, 33, 16, 3, 76,...
Literacy                                 [Literate, <NA>, Illiterate]
Current_student                         [NotStudying, Studying, <NA>]
EducationLevel      [106.0, 105.0, nan, 211.0, 214.0, 312.0, 212.0...
EmploymentStatus    [Housekeeper, Student, <NA>, Employed, IncomeW...
MaritalState             [Married, NotMarried, <NA>, Widow, Divorced]
dtype: object The number of Rows is : 152291
True
True
True
                   Year is :90                    


C:\Users\Mahsa\AppData\Local\Temp\ipykernel_14816\3135642001.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Rdata = pd.read_sql(f"SELECT * FROM R{year}P1", conn)
C:\Users\Mahsa\AppData\Local\Temp\ipykernel_14816\3135642001.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Udata = pd.read_sql(f"select * from U{year}P1", conn)


HHID                [2000021013, 2000129107, 2000069153, 200015305...
IndiNum             [01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 1...
RelationHead        [Head, Spouse, Child, Bro/Sis, s/d in low, Mom...
Sex                                              [Male, Female, <NA>]
Age                 [46.0, 41.0, 21.0, 16.0, 73.0, 62.0, 34.0, 44....
Literacy                                 [Literate, Illiterate, <NA>]
Current_student                         [NotStudying, Studying, <NA>]
EducationLevel      [106.0, 214.0, 324.0, 312.0, nan, 311.0, 104.0...
EmploymentStatus    [Employed, Housekeeper, Unemployed, Student, <...
MaritalState             [Married, NotMarried, <NA>, Widow, Divorced]
dtype: object The number of Rows is : 150540
False
False
False
                   Year is :91                    


...

HHID                [20001384220, 20013397029, 20001384225, 200013...
IndiNum             [01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 1...
RelationHead        [Head, Spouse, Child, Mom/Dad, Bro/Sis, Grandc...
Sex                                                    [Male, Female]
Age                 [35, 30, 4, 50, 47, 14, 12, 48, 43, 11, 40, 69...
Literacy                                 [Literate, <NA>, Illiterate]
Current_student                         [NotStudying, <NA>, Studying]
EducationLevel      [4.0, nan, 2.0, 1.0, 9.0, 6.0, 3.0, 5.0, 7.0, ...
EmploymentStatus    [Employed, Housekeeper, <NA>, Student, IncomeW...
MaritalState             [Married, <NA>, NotMarried, Widow, Divorced]
dtype: object The number of Rows is : 128914
True
True
False
                  Year is :1401                   


C:\Users\Mahsa\AppData\Local\Temp\ipykernel_14816\3135642001.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Rdata = pd.read_sql(f"SELECT * FROM R{year}P1", conn)
C:\Users\Mahsa\AppData\Local\Temp\ipykernel_14816\3135642001.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Udata = pd.read_sql(f"select * from U{year}P1", conn)


HHID                [21702576225, 21702576229, 21702576232, 217025...
IndiNum             [02, 03, 04, 05, 01, 06, 07, 08, 09, 10, 11, 1...
RelationHead        [Spouse, Child, Head, Mom/Dad, OtherRelative, ...
Sex                                              [Female, Male, <NA>]
Age                 [37, 13, 8, 3, 45, 41, 15, 12, 10, 60, 42, 24,...
Literacy                                 [Illiterate, Literate, <NA>]
Current_student                         [<NA>, NotStudying, Studying]
EducationLevel      [nan, 1.0, 2.0, 5.0, 4.0, 3.0, 6.0, 8.0, 7.0, ...
EmploymentStatus    [Employed, Housekeeper, <NA>, Student, IncomeW...
MaritalState             [Married, NotMarried, <NA>, Widow, Divorced]
dtype: object The number of Rows is : 126478
True
True
True
                  Year is :1402                   


C:\Users\Mahsa\AppData\Local\Temp\ipykernel_14816\3135642001.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Rdata = pd.read_sql(f"SELECT * FROM R{year}P1", conn)
C:\Users\Mahsa\AppData\Local\Temp\ipykernel_14816\3135642001.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Udata = pd.read_sql(f"select * from U{year}P1", conn)


HHID                [20001383933, 20001383935, 20001383938, 200013...
IndiNum             [01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 1...
RelationHead        [Head, Spouse, Child, Bro/Sis, Grandchild, s/d...
Sex                                                    [Male, Female]
Age                 [66, 59, 46, 42, 16, 2, 86, 69, 90, 82, 76, 51...
Literacy                                 [Literate, Illiterate, <NA>]
Current_student                         [NotStudying, <NA>, Studying]
EducationLevel      [1.0, nan, 2.0, 4.0, 3.0, 6.0, 5.0, 7.0, 9.0, ...
EmploymentStatus    [IncomeWithoutJob, Housekeeper, Employed, Unem...
MaritalState             [Married, NotMarried, <NA>, Widow, Divorced]
dtype: object The number of Rows is : 123857
True
True
True


In [77]:
years = list(range(89, 99))+ [99] + list(range(1400, 1403))
for year in years:
    df = pd.read_pickle(f"P1_{year}")
    if year == 93:
        df.loc[(df["HHID"]=="10207048807") & (df["IndiNum"]=='02'),'RelationHead']="Spouse"
    df["NumChild_U5"] = np.where(
        (df["RelationHead"]=="Child") & (df["Age"]<5), 1,0
    )
    df["NumChild_B5_7"] = np.where(
        (df["RelationHead"]=="Child") & ((df["Age"]>=5) & (df["Age"]<7)), 1,0
    )
    df["NumChild_B7_18"] = np.where(
        (df["RelationHead"]=="Child") & ((df["Age"]>=7) & (df["Age"]<18)), 1,0
    )
    df["NumChildOver18"] = np.where(
        (df["RelationHead"]=="Child") & (df["Age"]>=18), 1,0
    )
    df["HeadWoman"] = np.where(
        (df["IndiNum"]=="01") & (df["RelationHead"]=="Head") & (df["Sex"]=="Female"), 1, 0
    )
    df["Dep_Student"] = np.where(
        (df["NumChild_B7_18"]==1) & (df["Current_student"]=="NotStudying"), 1, 0
    )
    df["Num_Literate"] = np.where(
        df["Literacy"]=="Literate", 1, 0
    )
    df["NumSpouse"] = np.where(
        df["RelationHead"]=="Spouse", 1, 0
    )
    df["NumEmployed"] = np.where(
        df["EmploymentStatus"]=="Employed", 1, 0
    )
    df["NumOld"] = np.where(
        df["Age"]>= 65, 1, 0
    )
    df["NumMale"] = np.where(
        df["Sex"]=="Male", 1, 0
    )
    df["NumNotfirstR"] = np.where(
        df["RelationHead"].isin(['Head', 'Spouse', 'Child']), 0, 1
    )
    dfagg = df.groupby("HHID").aggregate(
        HHID = ("HHID", "first"),
        HHsize = ("HHID", "count"),
        AveAge = ("Age","mean"),
        NumChild_U5 = ("NumChild_U5","sum"),
        NumChild_B5_7 = ("NumChild_B5_7","sum"),
        NumChild_B7_18 = ("NumChild_B7_18","sum"),
        NumChildOver18 = ('NumChildOver18','sum'),
        HeadWoman = ("HeadWoman","sum"),
        Dep_Student = ('Dep_Student','sum'),
        Num_Literate = ('Num_Literate','sum'),
        NumSpouse = ('NumSpouse','sum'),
        NumEmployed = ('NumEmployed','sum'),
        NumOld = ('NumOld','sum'),
        NumMale = ('NumMale','sum'),
        NumNotfirstR = ("NumNotfirstR", "sum")
    ).reset_index(drop = True)
    df["MaleP"] = dfagg["NumMale"]/ dfagg["HHsize"]
    print(f'\n\n------------{year}------------\n')
    print(dfagg["HeadWoman"].max() ==1)
    print(dfagg["NumSpouse"].max())
    print(np.all(dfagg["NumChild_U5"]+dfagg["NumChild_B5_7"]+dfagg["NumChild_B7_18"]+dfagg["NumChildOver18"]+dfagg["NumNotfirstR"]+dfagg["NumSpouse"]+1==dfagg["HHsize"]))
    dfagg.to_pickle(fr"HHP1_{year}")




------------89------------

True
2
True


------------90------------

True
3
False


------------91------------

True
3
True


------------92------------

True
3
True


------------93------------

True
3
True


------------94------------

True
3
True


------------95------------

True
2
True


------------96------------

True
2
True


------------97------------

True
3
True


------------98------------

True
3
True


------------99------------

True
3
True


------------1400------------

True
3
True


------------1401------------

True
2
True


------------1402------------

True
2
True
